In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [3]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"

In [4]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTNCRF

In [6]:
model = BERTNCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3, nbest=len(data.train_ds.idx2label)-1)

build CRF...


In [7]:
from modules.train.train import NerLearner

In [8]:
num_epochs = 100

In [9]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTNCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [10]:
model.get_n_trainable_params()

299259

In [ ]:
learner.fit(epochs=num_epochs)

In [11]:
learner.load_model()

### Predict

In [12]:
from modules.data.bert_data import get_data_loader_for_predict

In [13]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [14]:
preds = learner.predict(dl)

In [15]:
from sklearn_crfsuite.metrics import flat_classification_report

In [16]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [17]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [18]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[5:], digits=4)

In [19]:
print(tokens_report)

              precision    recall  f1-score   support

       I_LOC     0.8765    0.7887    0.8303      1557
       I_PER     0.9598    0.9598    0.9598      2112
       I_ORG     0.7946    0.8078    0.8011      3865

   micro avg     0.8569    0.8464    0.8516      7534
   macro avg     0.8770    0.8521    0.8637      7534
weighted avg     0.8578    0.8464    0.8516      7534



In [20]:
from modules.analyze_utils.main_metrics import precision_recall_f1

In [21]:
results = precision_recall_f1(true_labels, pred_labels)

processed 56409 tokens with 7534 phrases; found: 7442 phrases; correct: 6377.

precision:  85.69%; recall:  84.64%; FB1:  85.16

	LOC: precision:  87.65%; recall:  78.87%; F1:  83.03 1401

	ORG: precision:  79.46%; recall:  80.78%; F1:  80.11 3929

	PER: precision:  95.98%; recall:  95.98%; F1:  95.98 2112


